In [1]:
import os

os.chdir("../../")

import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
from src.data_handling import ingestion

sns.set_theme(context="paper", font_scale=1.15, style="whitegrid")

In [2]:
ts_data = ingestion.read_combined().dropna()

d:\anaconda3\envs\ba\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
d:\anaconda3\envs\ba\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
d:\anaconda3\envs\ba\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
d:\anaconda3\envs\ba\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
X, y = ts_data.drop(columns=["consumption"]), ts_data["consumption"].values
X.shape, y.shape

((2250, 8), (2250,))

In [4]:
mm = MinMaxScaler()
ss = StandardScaler()

X_trans = ss.fit_transform(X)
y_trans = mm.fit_transform(y.reshape(-1, 1))

In [5]:
def create_sequences(X, y, n_steps_x, n_steps_y):
    Xs, ys = [], []
    for i in range(len(X)):
        # find the end of this pattern
        end_ix = i + n_steps_x
        out_end_ix = end_ix + n_steps_y
        # check if we are beyond the dataset
        if out_end_ix > len(X):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = X[i:end_ix], y[end_ix:out_end_ix]
        Xs.append(seq_x)
        ys.append(seq_y)
    return np.array(Xs), np.array(ys)

In [6]:
X_ss, y_mm = create_sequences(X_trans, y_trans, 100, 50)
X_ss.shape, y_mm.shape

((2101, 100, 8), (2101, 50, 1))

In [7]:
total_samples = X.shape[0]
train_test_cutoff = int(total_samples * 0.8)

X_train, X_test = X_ss[:train_test_cutoff], X_ss[train_test_cutoff:]
y_train, y_test = y_mm[:train_test_cutoff], y_mm[train_test_cutoff:]

print("Training shapes:", X_train.shape, y_train.shape)
print("Testing shapes:", X_test.shape, y_test.shape)

Training shapes: (1800, 100, 8) (1800, 50, 1)
Testing shapes: (301, 100, 8) (301, 50, 1)


In [27]:
X_train_tensors = Variable(torch.Tensor(X_train))
X_test_tensors = Variable(torch.Tensor(X_test))

y_train_tensors = Variable(torch.Tensor(y_train))
y_test_tensors = Variable(torch.Tensor(y_test))

# (number of sequences, number of intances (days) per sequence, number of features in each instance)
print("Training Shape:", X_train_tensors.shape, y_train_tensors.shape)
print("Testing Shape:", X_test_tensors.shape, y_test_tensors.shape)

Training Shape: torch.Size([1800, 100, 8]) torch.Size([1800, 50, 1])
Testing Shape: torch.Size([301, 100, 8]) torch.Size([301, 50, 1])


In [29]:
class LSTM(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super().__init__()

        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size

        # LSTM model
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=0.2,
        )

        # Fully connected layer
        self.fc_1 = nn.Linear(hidden_size, 128)

        # Fully connected last layer
        self.fc_2 = nn.Linear(128, num_classes)

        self.relu = nn.ReLU()

    def forward(self, x):
        # hidden state
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))

        # cell state
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))

        # Propagate input through LSTM
        output, (hn, cn) = self.lstm(
            x, (h_0, c_0)
        )  # (input, hidden, and internal state)
        hn = hn.view(-1, self.hidden_size)
        out = self.relu(hn)
        out = self.fc_1(out)  # first dense
        out = self.relu(out)  # relu
        out = self.fc_2(out)  # final output
        return out

In [30]:
def training_loop(n_epochs, lstm, optimiser, loss_fn, X_train, y_train, X_test, y_test):

    for epoch in range(n_epochs):
        lstm.train()
        outputs = lstm.forward(X_train)  # forward pass
        optimiser.zero_grad()  # calculate the gradient, manually setting to 0

        # obstain the loss function
        loss = loss_fn(outputs, y_train)
        loss.backward()  # calculate the loss of the loss function
        optimiser.step()  # improve from loss, i.e backdrop

        # test loss
        lstm.eval()
        test_preds = lstm(X_test)
        test_loss = loss_fn(test_preds, y_test)

        if epoch % 100 == 0:
            print(
                f"Epoch: {epoch}, train loss: {loss.item()}, test loss: {test_loss.item()}"
            )

In [ ]:
n_epochs = 1000
learning_rate = 0.001

input_size = 8
hidden_size = 4
num_layers = 1

num_classes = 50

lstm = LSTM(num_classes, input_size, hidden_size, num_layers)